__Вводные данные:__

Дана выгрузка данных пользователей в виде когортного анализа. Стоимость подписки, которую приобретали пользователи мобильного приложения, – 9,99 $.

__Задача:__

Необходимо рассчитать прогнозный LTV24 (сколько нам принесет пользователь на протяжении 24-х месяцев использования приложения) на основании имеющихся данных. Следует учитывать, что с покупки подписки 30 % от стоимости подписки забирает Apple в первый год, 15 % - во второй.

---

In [166]:
import pandas as pd

In [167]:
cohort = pd.read_excel('result.xlsx', sheet_name=2, usecols='B:AA', nrows=23)
cohort.fillna(0, inplace=True)

In [168]:
cohort

,Date,Trial,Month 1,Month 2,Month 3,Month 4,Month 5,Month 6,Month 7,Month 8,...,Month 15,Month 16,Month 17,Month 18,Month 19,Month 20,Month 21,Month 22,Month 23,Month 24
0,2017-04-01,1501,711,437.0,286.0,206.0,142.0,116.0,95.0,78.0,...,32.0,27.0,21.0,18.0,15.0,14.0,11.0,9.0,0.0,0.0
1,2017-05-01,1253,544,338.0,246.0,178.0,138.0,109.0,89.0,71.0,...,21.0,19.0,16.0,15.0,15.0,13.0,13.0,0.0,0.0,0.0
2,2017-06-01,1226,522,319.0,228.0,169.0,133.0,99.0,82.0,72.0,...,27.0,26.0,25.0,23.0,21.0,19.0,0.0,0.0,0.0,0.0
3,2017-07-01,1484,538,297.0,201.0,147.0,112.0,85.0,77.0,64.0,...,32.0,30.0,23.0,20.0,18.0,0.0,0.0,0.0,0.0,0.0
4,2017-08-01,2009,775,449.0,317.0,257.0,198.0,170.0,148.0,126.0,...,51.0,41.0,38.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2017-09-01,2132,940,580.0,384.0,289.0,225.0,177.0,146.0,125.0,...,44.0,39.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2017-10-01,1231,467,279.0,196.0,148.0,103.0,84.0,62.0,53.0,...,21.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2017-11-01,1035,392,194.0,128.0,118.0,80.0,65.0,65.0,48.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,2017-12-01,511,241,127.0,94.0,87.0,57.0,51.0,47.0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2018-01-01,2167,756,439.0,311.0,244.0,186.0,140.0,119.0,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [169]:
cohort_active = cohort.copy(deep=True)

# расчёт процента продления подписки (то же, что Retention Rate)
for i in range(22):
    for j in range(2, 26):
        cohort_active.iloc[i, j] = cohort_active.iloc[i, j] / cohort_active.iloc[i, 1]

In [170]:
cohort_active

,Date,Trial,Month 1,Month 2,Month 3,Month 4,Month 5,Month 6,Month 7,Month 8,...,Month 15,Month 16,Month 17,Month 18,Month 19,Month 20,Month 21,Month 22,Month 23,Month 24
0,2017-04-01,1501,0.473684,0.291139,0.190540,0.137242,0.094604,0.077282,0.063291,0.051965,...,0.021319,0.017988,0.013991,0.011992,0.009993,0.009327,0.007328,0.005996,0.0,0.0
1,2017-05-01,1253,0.434158,0.269753,0.196329,0.142059,0.110136,0.086991,0.071030,0.056664,...,0.016760,0.015164,0.012769,0.011971,0.011971,0.010375,0.010375,0.000000,0.0,0.0
2,2017-06-01,1226,0.425775,0.260196,0.185971,0.137847,0.108483,0.080750,0.066884,0.058728,...,0.022023,0.021207,0.020392,0.018760,0.017129,0.015498,0.000000,0.000000,0.0,0.0
3,2017-07-01,1484,0.362534,0.200135,0.135445,0.099057,0.075472,0.057278,0.051887,0.043127,...,0.021563,0.020216,0.015499,0.013477,0.012129,0.000000,0.000000,0.000000,0.0,0.0
4,2017-08-01,2009,0.385764,0.223494,0.157790,0.127924,0.098556,0.084619,0.073668,0.062718,...,0.025386,0.020408,0.018915,0.017919,0.000000,0.000000,0.000000,0.000000,0.0,0.0
5,2017-09-01,2132,0.440901,0.272045,0.180113,0.135553,0.105535,0.083021,0.068480,0.058630,...,0.020638,0.018293,0.017355,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
6,2017-10-01,1231,0.379366,0.226645,0.159220,0.120227,0.083672,0.068237,0.050366,0.043054,...,0.017059,0.012185,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
7,2017-11-01,1035,0.378744,0.187440,0.123671,0.114010,0.077295,0.062802,0.062802,0.046377,...,0.019324,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
8,2017-12-01,511,0.471624,0.248532,0.183953,0.170254,0.111546,0.099804,0.091977,0.064579,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
9,2018-01-01,2167,0.348869,0.202584,0.143516,0.112598,0.085833,0.064605,0.054915,0.043378,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [171]:
# расчёт среднего процента продления подписки по месяцам
agg_cohort_active = cohort_active.copy(deep=True)
agg_cohort_active.drop(['Date', 'Trial'], axis=1, inplace=True)
mean_cohort_active = agg_cohort_active.mean()

In [172]:
mean_cohort_active

Month 1     0.376404
Month 2     0.199256
Month 3     0.130434
Month 4     0.101948
Month 5     0.069772
Month 6     0.055140
Month 7     0.046631
Month 8     0.034584
Month 9     0.028572
Month 10    0.024201
Month 11    0.018807
Month 12    0.015547
Month 13    0.012375
Month 14    0.009997
Month 15    0.007458
Month 16    0.005703
Month 17    0.004496
Month 18    0.003369
Month 19    0.002328
Month 20    0.001600
Month 21    0.000805
Month 22    0.000273
Month 23    0.000000
Month 24    0.000000
dtype: float64

In [173]:
# расчёт дохода относительно вовлечённости: 6.99 $ в первый год, 8.49 $ во второй год
mean_cohort_active[:12] = mean_cohort_active[:12] * 6.99
mean_cohort_active[12:] = mean_cohort_active[12:] * 8.49
revenue_24 = mean_cohort_active

In [174]:
revenue_24

Month 1     2.631061
Month 2     1.392802
Month 3     0.911731
Month 4     0.712617
Month 5     0.487703
Month 6     0.385429
Month 7     0.325954
Month 8     0.241743
Month 9     0.199717
Month 10    0.169166
Month 11    0.131458
Month 12    0.108672
Month 13    0.105062
Month 14    0.084878
Month 15    0.063317
Month 16    0.048416
Month 17    0.038174
Month 18    0.028604
Month 19    0.019767
Month 20    0.013584
Month 21    0.006832
Month 22    0.002314
Month 23    0.000000
Month 24    0.000000
dtype: float64

In [175]:
# расчёт LTV за 24 месяца
round(sum(revenue_24), 2)

8.11

Пользователь, пользуясь приложением 2 года, принесёт компании чуть больше __8 $__.